<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/tools/CHE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import re
import requests
import string
from collections import deque
import numpy as np
from itertools import combinations

two_pi=2*np.pi
one_third = 1/3


extract_single_props = {'Molecular Weight' : 'Mw',
                        'Critical Temperature' : 'Tc',
                        'Critical Pressure' : 'Pc',
                        'Critical Volume' : 'Vc',
                        'Acentric factor' : 'w',
                        'Normal boiling point' : 'Tb',
                        'IG heat of formation' : 'HfIG',
                        'IG Gibbs of formation' : 'GfIG',
                        'Heat of vaporization' : 'HvapNB'}

extract_coeff_props={'Vapor Pressure' : 'Pvap',
                     'Ideal Gas Heat Capacity':'CpIG',
                     'Liquid Heat Capacity' : 'CpL',
                     'Solid Heat Capacity' : 'CpS',
                     'Heat of Vaporization' : 'Hvap',
                     'Liquid Density' : 'rhoL'}

extract_poly_coeff_props={'Polynomial Ideal Gas Heat Capacity (cal/mol-K)':'polyCpIG'}

base_url = 'https://raw.githubusercontent.com/profteachkids/chetools/master/data/'

BIP_file = 'https://raw.githubusercontent.com/profteachkids/chetools/master/data/BinaryNRTL.txt'

class Props():
    def __init__(self,comps, get_NRTL=True, base_url=base_url, extract_single_props=extract_single_props,
                 extract_coeff_props=extract_coeff_props, BIP_file=BIP_file, suffix='Props.txt'):

        comps = [comps] if isinstance(comps,str) else comps
        self.N_comps = len(comps)

        id_pat = re.compile(r'ID\s+(\d+)')
        formula_pat = re.compile(r'Formula:\s+([A-Z0-9]+)')
        single_props_pat = re.compile(r'^\s+([\w \/.]+?)\s+:\s+([-.0-9e+]+) +([-\w/.()*]*) *$', re.MULTILINE)
        coeffs_name_pat = re.compile(r"([\w ]+)\s[^\n]*?Equation.*?Coeffs:([- e\d.+]+)+?", re.DOTALL)
        coeffs_pat = re.compile(r'([-\de.+]+)')
        poly_coeffs_pat = re.compile(r"([- \/'()A-Za-z]*)\n Coefficients: +([-+e\d.+]*)\n* *([-+e\d.+]*)\n* *([-+e\d.+]*)\n* *([-+e\d.+]*)\n* *([-+e\d.+]*)\n* *([-+e\d.+]*)\n* *([-+e\d.+]*)")

        props_deque=deque()
        for comp in comps:
            res = requests.get(base_url+comp + suffix)
            if res.status_code != 200:
                raise ValueError(f'{comp} - no available data')
            text=res.text
            props={'Name': comp}
            units={}
            props['ID']=id_pat.search(text).groups(1)[0]
            props['Formula']=formula_pat.search(text).groups(1)[0]
            single_props = dict((item[0], item[1:]) for item in single_props_pat.findall(text))
            for k,v in extract_single_props.items():
                props[v]=float(single_props[k][0])
                units[v]=single_props[k][1]
                props[v] = props[v]*2.20462*1055.6 if units[v]=='Btu/lbmol' else props[v]
                props[v] = props[v]*6894.76 if units[v]=='psia' else props[v]
                props[v] = (props[v]-32)*5/9 + 273.15 if units[v] =='F' else props[v]

            coeffs_name_strings = dict(coeffs_name_pat.findall(text))
            for k,v in extract_coeff_props.items():
                coeffs = coeffs_pat.findall(coeffs_name_strings[k])
                for letter, value in zip(string.ascii_uppercase,coeffs):
                    props[v+letter]=float(value)
            poly_props = dict([(item[0], item[1:]) for item in poly_coeffs_pat.findall(text)])
            for k,v in extract_poly_coeff_props.items():
                for letter, value in zip(string.ascii_uppercase,poly_props[k]):
                    if value == '':
                        break
                    props[v+letter]=float(value)


            props_deque.append(props)

        for prop in props_deque[0].keys():
            # if self.N_comps>1:
            values = np.array([comp[prop] for comp in props_deque])
            # else:
            #     values = props_deque[0][prop]
            setattr(self,prop,values)


        # kmol to mol
        self.Vc = self.Vc/1000.
        self.HfIG = self.HfIG/1000.
        self.HfL = self.HfIG - self.Hvap(298.15)
        self.GfIG = self.GfIG/1000.
        self.HvapNB=self.HvapNB/1000.

        if (self.N_comps > 1) and get_NRTL:
            text = requests.get(BIP_file).text

            comps_string = '|'.join(self.ID)
            id_name_pat = re.compile(r'^\s+(\d+)[ ]+(' + comps_string +')[ ]+[A-Za-z]',re.MULTILINE)
            id_str = id_name_pat.findall(text)

            #maintain order of components
            id_dict = {v:k for k,v in id_str}
            # list of comp IDs with BIP data
            id_str = [id_dict.get(id, None) for id in self.ID]
            id_str_indices=list(filter(lambda x: False if x[0] is None else True, zip(id_str,range(len(id_str)))))
            id_str,indices=list(zip(*id_str_indices))
            comb_strs = combinations(id_str,2)
            comb_indices = combinations(indices,2)
            self.NRTL_A, self.NRTL_B, self.NRTL_C, self.NRTL_D, self.NRTL_alpha = np.zeros((5, self.N_comps,self.N_comps))
            start=re.search(r'Dij\s+Dji',text).span()[0]

            for comb_id, comb_index in zip(comb_strs, comb_indices):
                comb_str = '|'.join(comb_id)
                comb_values_pat = re.compile(r'^[ ]+(' + comb_str +
                                             r')[ ]+(?:' + comb_str + r')(.*)$', re.MULTILINE)


                match = comb_values_pat.search(text[start:])
                if match is not None:
                    first_id, values = match.groups(1)
                    #if matched order is flipped, also flip indices
                    if first_id != comb_id[0]:
                        comb_index = (comb_index[1],comb_index[0])
                    bij, bji, alpha, aij, aji, cij, cji, dij, dji  = [float(val) for val in values.split()]
                    np.add.at(self.NRTL_B, comb_index, bij)
                    np.add.at(self.NRTL_B, (comb_index[1],comb_index[0]), bji)
                    np.add.at(self.NRTL_A, comb_index, aij)
                    np.add.at(self.NRTL_A, (comb_index[1],comb_index[0]), aji)
                    np.add.at(self.NRTL_C, comb_index, cij)
                    np.add.at(self.NRTL_C, (comb_index[1],comb_index[0]), cji)
                    np.add.at(self.NRTL_D, comb_index, dij)
                    np.add.at(self.NRTL_D, (comb_index[1],comb_index[0]), dji)
                    np.add.at(self.NRTL_alpha, comb_index, alpha)
                    np.add.at(self.NRTL_alpha, (comb_index[1],comb_index[0]), alpha)



    def Pvap(self,T):
        T=np.atleast_1d(np.squeeze(np.asarray(T)))
        return np.squeeze(np.exp(self.PvapA[None,:] + self.PvapB[None,:]/T[:,None] + self.PvapC[None,:]*np.log(T[:,None]) +
                       self.PvapD[None,:]*np.power(T[:,None],self.PvapE[None,:])))


    def CpIG(self, T):
        T=np.atleast_1d(np.squeeze(np.asarray(T)))
        CpIGCT = self.CpIGC[None,:]/T[:,None]
        CpIGET = self.CpIGE[None,:]/T[:,None]
        return np.squeeze(self.CpIGA[None,:] + self.CpIGB[None,:]*(CpIGCT/np.sinh(CpIGCT))**2 +
                self.CpIGD[None,:]*(CpIGET/np.cosh(CpIGET))**2)


    def deltaHsensIGpoly(self, T):
        T=np.atleast_1d(np.squeeze(np.asarray(T)))
        return np.squeeze(T[:,None] * (self.polyCpIGA[None,:] + T[:,None]* (self.polyCpIGB[None,:] / 2 + T[:,None] * (self.polyCpIGC[None,:] / 3 + 
            T[:,None] * (self.polyCpIGD[None,:] / 4 + T[:,None]* (self.polyCpIGE[None,:] / 5 + T[:,None]*self.polyCpIGF[None,:]/6)))))*4.184)


    def HIGpoly(self, nV, T):
        nV=np.atleast_2d(nV).reshape(-1,self.N_comps)
        return np.squeeze(np.sum(nV * (self.HfIG + self.deltaHsensIGpoly(T) - self.deltaHsensIGpoly(298.15)),axis=-1))



    def deltaHsensIG(self, T):
        T=np.atleast_1d(np.squeeze(np.asarray(T)))
        return np.squeeze(self.CpIGA[None,:]*T[:,None] + self.CpIGB[None,:] * self.CpIGC[None,:]/np.tanh(self.CpIGC[None,:]/T[:,None]) - 
            self.CpIGD[None,:] * self.CpIGE[None,:] * np.tanh(self.CpIGE[None,:]/T[:,None]))/1000


    def HIG(self, nV, T):
        nV=np.atleast_2d(nV).reshape(-1,self.N_comps)
        return np.squeeze(np.sum(nV*(self.HfIG + self.deltaHsensIG(T) - self.deltaHsensIG(298.15)),axis=-1))



    def Hvap(self, T):
        T=np.atleast_1d(np.squeeze(np.asarray(T)))
        Tr = T[:,None]/np.atleast_1d(self.Tc)[None,:]
        return np.squeeze(self.HvapA[None,:]*np.power(1-Tr[:,None] , self.HvapB[None,:] + (self.HvapC[None,:]
            +(self.HvapD[None,:]+self.HvapE[None,:]*Tr[:,None] )*Tr[:,None] )*Tr[:,None] ))/1000.



    def deltaHsensL(self, T):
        T=np.atleast_1d(np.squeeze(np.asarray(T)))
        return np.squeeze(T[:,None] * (self.CpLA[None,:] + T[:,None] * (self.CpLB[None,:]/ 2 + T[:,None] * (self.CpLC[None,:] / 3 + T[:,None] *
             (self.CpLD[None,:] / 4 + self.CpLE[None,:] / 5 * T[:,None])))))/1000.


    def Hv(self, nV, T):
        nV=np.atleast_2d(nV).reshape(-1,self.N_comps)
        return np.squeeze(self.Hl(nV, T) + np.sum(nV*self.Hvap(T),axis=-1))


    def Hl(self, nL, T):
        T=np.atleast_1d(np.squeeze(np.asarray(T)))
        nL=np.atleast_2d(nL).reshape(-1,self.N_comps)
        return np.squeeze(np.sum(nL*(self.HfL + self.deltaHsensL(T) - self.deltaHsensL(298.15)),axis=-1))


    def rhol(self, T):
        T=np.atleast_1d(np.squeeze(np.asarray(T)))
        return np.squeeze(self.rhoLA[None,:] / np.power(self.rhoLB[None,:], 1+ np.power((1.-T[:,None]/self.rhoLC[None,:]),self.rhoLD[None,:])) *self.Mw[None,:])
        

    def NRTL_gamma(self, x, T):
        x=np.atleast_2d(x).reshape(-1,self.N_comps)
        T=np.atleast_1d(np.squeeze(np.asarray(T)))
        tau = (self.NRTL_A[None,:,:] + self.NRTL_B[None,:,:] / T[:,None,None] + self.NRTL_C[None,:,:] * np.log(T[:,None,None]) +
               self.NRTL_D[None,:,:] * T[:,None,None])

        G = np.exp(-self.NRTL_alpha[None,:,:] * tau)
        xG=x[:,None,:] @ G
        xtauGdivxG = (x[:,None,:]@ (tau*G)/ xG)

        lngamma = np.squeeze(xtauGdivxG) +  np.squeeze(((G*(tau - xtauGdivxG))/xG) @x[:,:,None])
        return np.exp(lngamma)


def qtox(q):
    q=np.atleast_1d(q)
    xm1 = np.exp(q)/(1+np.sum(np.exp(q)))
    return np.concatenate((xm1, np.atleast_1d(1.-np.sum(xm1))))


def xtoq(x):
    x=np.atleast_1d(x)
    return np.log(x[:-1]) + np.log(1.+ (1. - x[-1])/x[-1])


def cube_roots(a,b,c):
    Q=(a**2 - 3*b)/9
    R = (2*a**3 - 9*a*b + 27*c)/54
    if R**2 < Q**3:
        theta = np.arccos(R/Q**1.5)
        roots = -2*Q**0.5 * np.cos([ theta/3, (theta+2*np.pi)/3, (theta-2*np.pi)/3]) - a/3
        return np.sort(roots)
    else:
        A = - np.sign(R)*(np.abs(R) + (R**2 - Q**3)**0.5   )**(1/3)
        B = 0. if A==0 else Q/A
        return (A+B)-a/3